In [ ]:
from IPython.utils import capture
with capture.capture_output() as cap:
  #
  from google.colab import drive
  drive.mount('/content/drive')
  #
  %cd /content
  !mkdir /content/drive/MyDrive/dataset
  !mkdir /content/drive/MyDrive/train
  !mkdir /content/log
  !mkdir -p /root/.cache/huggingface/accelerate/
  #
  !curl -Lo /content/ae.safetensors https://huggingface.co/josemerinom/flux/resolve/main/ae.safetensors
  !curl -Lo /content/clip_l.safetensors https://huggingface.co/josemerinom/flux/resolve/main/clip_l.safetensors
  !curl -Lo /content/flux1-dev-fp8.safetensors https://huggingface.co/josemerinom/flux/resolve/main/flux1-dev-fp8.safetensors
  !curl -Lo /content/t5xxl_fp8_e4m3fn.safetensors https://huggingface.co/josemerinom/flux/resolve/main/t5xxl_fp8_e4m3fn.safetensors
  #
  !curl -Lo /content/default_config.yaml https://huggingface.co/josemerinom/flux/resolve/main/default_config.yaml
  !curl -Lo /content/zeroLR.zip https://huggingface.co/josemerinom/flux/resolve/main/zeroLR_sd3_may01.zip
  !unzip -u /content/zeroLR.zip
  !cp -r /content/default_config.yaml /root/.cache/huggingface/accelerate/
  #
  !pip uninstall -y peft
  !pip uninstall -y accelerate
  !pip uninstall -y torch
  !pip uninstall -y torchvision
  !pip uninstall -y torchaudio
  #
  !pip install torch==2.4.0 --index-url https://download.pytorch.org/whl/cu124
  !pip install torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu124
  !pip install torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124
  #
  !pip install accelerate==0.33.0
  !pip install transformers==4.44.0
  !pip install diffusers[torch]==0.25.0
  !pip install ftfy==6.1.1
  !pip install albumentations==1.3.0
  !pip install opencv-python==4.8.1.78
  !pip install einops==0.7.0
  !pip install pytorch-lightning==1.9.0
  !pip install bitsandbytes==0.44.0
  !pip install lion-pytorch==0.0.6
  !pip install schedulefree==1.4
  !pip install pytorch-optimizer==3.5.0
  !pip install prodigy-plus-schedule-free==1.9.0
  !pip install prodigyopt==1.1.2
  !pip install tensorboard
  !pip install safetensors==0.4.4
  !pip install altair==4.2.2
  !pip install easygui==0.98.3
  !pip install toml==0.10.2
  !pip install voluptuous==0.13.1
  !pip install huggingface-hub==0.24.5
  !pip install imagesize==1.4.1
  !pip install numpy<=2.0
  !pip install rich==13.7.1
  !pip install sentencepiece==0.2.0
  !pip install deepspeed==0.16.7
  !pip install xformers==0.0.27.post2
  #
  %cd /content/zeroLR
  !pip install -e .
  #
print("done")

In [ ]:
%cd /content/zeroLR
#
def lora(name,arg):
  l_name=name
  l_arg=arg
  print(l_arg)
  !accelerate launch --num_processes=1 --num_machines=1 --num_cpu_threads_per_process=1 flux_train_network.py \
    --adaptive_noise_scale=0 \
    --ae="/content/ae.safetensors" \
    --apply_t5_attn_mask \
    --blocks_to_swap=18 \
    --cache_latents \
    --caption_extension=".txt" \
    --clip_l="/content/clip_l.safetensors" \
    --clip_skip=1 \
    --console_log_simple \
    --discrete_flow_shift=3 \
    --fp8_base \
    --gradient_accumulation_steps=1 \
    --gradient_checkpointing \
    --guidance_scale=1 \
    --huber_c=0.1 \
    --huber_schedule="snr" \
    --ip_noise_gamma=0 \
    --keep_tokens=0 \
    --learning_rate=2e-4 \
    --logging_dir="/content/log" \
    --loss_type="l2" \
    --lr_scheduler="constant" \
    --lr_scheduler_num_cycles=1 \
    --lr_scheduler_power=1 \
    --lr_warmup_steps=0 \
    --max_data_loader_n_workers=0 \
    --max_grad_norm=1 \
    --max_train_steps=2000 \
    --min_snr_gamma=0 \
    --mixed_precision="bf16" \
    --model_prediction_type="raw" \
    --network_alpha=16 \
    --network_dim=16 \
    --network_dropout=0 \
    --network_module=networks.lora_flux \
    --network_train_unet_only \
    --noise_offset=0 \
    --optimizer_args "betas=(0.9, 0.999)" "eps=1e-8" "weight_decay=0.01" \
    --optimizer_type="AdamW8bit" \
    --output_dir="/content/drive/MyDrive/train" \
    --output_name=$l_name \
    --pretrained_model_name_or_path="/content/flux1-dev-fp8.safetensors" \
    --prior_loss_weight=1 \
    --resolution=512 \
    --save_every_n_epochs=1 \
    --save_model_as="safetensors" \
    --save_precision="bf16" \
    --save_state \
    --scale_weight_norms=0 \
    --seed=1 \
    --sigmoid_scale=1 \
    --t5xxl="/content/t5xxl_fp8_e4m3fn.safetensors" \
    --t5xxl_max_token_length=512 \
    --text_encoder_lr=0 \
    --timestep_sampling="sigmoid" \
    --train_batch_size=2 \
    --train_data_dir="/content/drive/MyDrive/dataset" \
    --unet_lr=2e-4 \
    --xformers \
    $l_arg
    #
lora('TEST_PART_A','--initial_epoch=1')
#lora('TEST_PART_B','--initial_epoch=19 --resume="/content/drive/MyDrive/train/TEST_PART_A-000018-state"')
#lora('TEST_PART_C','--initial_epoch=35 --resume="/content/drive/MyDrive/train/TEST_PART_B-000034-state"')

In [ ]:
#To continue training add the parameter:
#--resume and indicate the last stored state. example: --resume="/content/drive/MyDrive/train/LoraName-xx-state"
#--initial_epoch=xx It is used to indicate at what number it will start saving, if the last state is 8, it will start at 9 when it continues training.